In [2]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from pybliometrics.scopus import ScopusSearch
from bs4 import BeautifulSoup
import requests, json
from docx import Document
from docx.shared import Inches

In [3]:
from my_apikeys import APIKeys

In [ ]:
titles = [
    "First-Principles-Based Machine-Learning Molecular Dynamics for Crystalline Polymers with van der Waals interactions",
    "Optimizing machine learning models for granular NdFeB magnets by very fast simulated annealing",
    "Resonant vortex-core reversal in magnetic nano-spheres as robust mechanism of efficient energy absorption and emission",
    "Temperature effect on vortex-core reversals in magnetic nanodots",
    "Resonantly excited precession motion of three-dimensional vortex core in magnetic nanospheres", 
    "Self-assembled magnetic nanospheres with three-dimensional magnetic vortex",
    "Magnetic Microstructure of Rolled-Up Single-Layer Ferromagnetic Nanomembranes",
    
]

In [27]:
# author id of "Jehyun Lee"
author_id = "55969873500"

author_r = requests.get(f"http://api.elsevier.com/content/author?author_id={author_id}&view=metrics",
                        headers={'Accept':'application/json', 'X-ELS-APIKey': APIKeys[-2]})
author_data = eval(json.dumps(author_r.json(), sort_keys=True, indent=4, separators=(',', ': ')))

h_index = author_data['author-retrieval-response'][0]["h-index"]
citedby_count_total = author_data['author-retrieval-response'][0]["coredata"]['cited-by-count']
document_count_total = author_data['author-retrieval-response'][0]["coredata"]['document-count']

print(h_index, citedby_count_total, document_count_total)

In [33]:
title = "First-Principles-Based Machine-Learning Molecular Dynamics for Crystalline Polymers with van der Waals interactions"
s_sample = ScopusSearch(f"TITLE ({title})").results[0]
list(zip(s_sample.author_names.split(";"), s_sample.author_ids.split(";")))

[('Hong, Sung Jun', '57221590144'),
 ('Chun, Hoje', '57195466596'),
 ('Lee, Jehyun', '55969873500'),
 ('Kim, Byung Hyun', '55502655900'),
 ('Seo, Min Ho', '57220796436'),
 ('Kang, Joonhee', '56076915900'),
 ('Han, Byungchan', '10639592000')]

In [34]:
title = "Resonantly excited precession motion of three-dimensional vortex core in magnetic nanospheres"
s_sample = ScopusSearch(f"TITLE ({title})").results[0]
list(zip(s_sample.author_names.split(";"), s_sample.author_ids.split(";")))

[('Kim, Sang Koog', '7601587994'),
 ('Yoo, Myoung Woo', '35738330300'),
 ('Lee, Jehyun', '16029235100'),
 ('Lee, Ha Youn', '56206010000'),
 ('Lee, Jae Hyeok', '57206731799'),
 ('Gaididei, Yuri', '7005459050'),
 ('Kravchuk, Volodymyr P.', '15757587900'),
 ('Sheka, Denis D.', '6701719591')]

In [ ]:
author_pubs_ = requests.get(f"http://api.elsevier.com/content/search/scopus?query=AU-ID({author_id})&field=dc:identifier&count=100",
                               headers={'Accept':'application/json',
                                        'X-ELS-APIKey': APIKeys[-3]})
author_pubs = author_pubs_.json()
scopus_ids = [p['dc:identifier'] for p in author_pubs['search-results']["entry"]]

In [18]:
len(author_pubs['search-results']["entry"])

100

In [19]:
def get_scopus_info(SCOPUS_ID):
    url = ("http://api.elsevier.com/content/abstract/scopus_id/"
          + SCOPUS_ID
          + "?field=authors,title,publicationName,volume,issueIdentifier,"
          + "prism:pageRange,coverDate,article-number,doi,citedby-count,prism:aggregationType")
    resp = requests.get(url,
                    headers={'Accept':'application/json',
                             'X-ELS-APIKey': APIKeys[-3]})
    results = json.loads(resp.text.encode('utf-8'))
    
    authors = ', '.join([au['ce:indexed-name'] for au in results['abstracts-retrieval-response']['authors']['author']])
    title = results['abstracts-retrieval-response']['coredata']['dc:title']
    pubtype = results['abstracts-retrieval-response']['coredata']['prism:aggregationType']
    journal = results['abstracts-retrieval-response']['coredata']['prism:publicationName']
    volume = results['abstracts-retrieval-response']['coredata']['prism:volume']
    articlenum = (results['abstracts-retrieval-response']['coredata'].get('prism:pageRange') or
                  results['abstracts-retrieval-response']['coredata'].get('article-number'))
    date = results['abstracts-retrieval-response']['coredata']['prism:coverDate']
    doi = 'doi:' + results['abstracts-retrieval-response']['coredata']['prism:doi']
    cites = int(results['abstracts-retrieval-response']['coredata']['citedby-count'])
    
    return pubtype, f'{authors}, {title}, {journal}, {volume}, {articlenum}, ({date}). {doi} (cited {cites} times).\n'

In [20]:
pub_no = 1
pubs = []
for sid in scopus_ids:
    try:
        pub_type, pub_info = get_scopus_info(sid)
        if pub_type == 'Journal':
            pubs.append(f"{pub_info}")
            pub_no += 1
    except:
        pass

print(pub_no)

94


In [24]:
# export to .docx file
document = Document()
document.add_heading(f"Jehyun Lee({len(pubs)} publications)\n")

for pub in pubs:
    document.add_paragraph(f'{pub}', style='List Number')
document.save(f'my_CV.docx')